#### Clarusway Python

* [Instructor Landing Page](landing_page.ipynb)
* <a href="https://colab.research.google.com/github/4dsolutions/clarusway_data_analysis/blob/main/basic_python/sandbox_week_07.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
* [![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/4dsolutions/clarusway_data_analysis/blob/main/basic_python/sandbox_week_07.ipynb)

# Sandbox 7: Completing Python

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/54172991810/in/album-72177720296706479" title="Recommended Reading"><img src="https://live.staticflickr.com/65535/54172991810_77931ed545.jpg" width="500" height="226" alt="Recommended Reading"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

[Think Python: Chapter 14](https://allendowney.github.io/ThinkPython/chap14.html) (free online)

[Advanced Python](https://docs.google.com/presentation/d/1Dp9qhRNSPrSl3u1nhNk1hjUzBDd_fVG662TAaLp8EoY/edit?usp=sharing) (Google slide deck)

[thekirbster demo website](https://thekirbster.pythonanywhere.com) (uses Flask framework)

## Reviewing Exceptions

In [51]:
class PantsOnFire(Exception):
    pass

def test_me():
    try:
        print("I am testing you now")
        1/0
    except:
        print("bad!")
        raise PantsOnFire("Don't worry though!")

try:
    assert 1 == 2
    # test_me() 
except PantsOnFire as e:
    print(e)
except AssertionError as e:
    print("assert error")
except Exception as e:
    print(e)
else:
    print("Never got here")
finally:
    print("That was fun")

assert error
That was fun


## Composing Functions

In [2]:
def compose(f, g):
    def h(x):
        return f(g(x))
    return h

def m(s):
    return s + "s"

def n(s):
    return s.upper()

funk1 = compose(m, n)
funk2 = compose(n, m)

In [3]:
funk1("Hello")

'HELLOs'

In [4]:
funk2("Hello")

'HELLOS'

In [5]:
h = compose(lambda x: 2*x, lambda y: y+2)

In [6]:
h(10)

24

In [7]:
(lambda x: 2*x)((lambda y: y+2)(10))

24

In [8]:
h = compose(lambda y: y+2, lambda x: 2*x)

In [9]:
h(10)

22

In [10]:
(lambda y: y+2)((lambda x: 2*x)(10))

22

In [33]:
class Compose:

    def __init__(self, f):
        self.f = f
    def __call__(self, x):
        return self.f(x)
    def __mul__(self, g):
        return Compose(lambda x: self.f(g(x))) # creates new object with lambda

funk1 = Compose(funk1) # funk1: upper then add s
funk2 = Compose(funk2) # funk2: add s then upper

funk3 = funk1 * funk2  # add s then upper, then upper and add s
funk4 = funk2 * funk1  # upper then add s, then add s then upper

In [34]:
funk3("bravo")

'BRAVOSs'

In [35]:
funk4("bravo")

'BRAVOSS'

Exercise: use Compose class with simple f, g of your own making. Show (f * g)(x) and (g * f)(x)

## Decorator Syntax

Take a look at this syntax:

```python
    funk1 = Compose(funk1) # funk1: upper then add s
    funk2 = Compose(funk2) # funk2: add s then upper
```

The input and output have the same name, but a transformation is happening. funk1 is going in as a function but coming out as a Compose type instance.

When you want to eat a callable right as it's being defined, and return a transformed version with the same name, Python offers "decorator syntax" (@).

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/54174817700/in/dateposted/" title="Abduction Ahead"><img src="https://live.staticflickr.com/65535/54174817700_559890a22d_w.jpg" width="400" height="400" alt="Abduction Ahead"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

You might think of "being decorated" as akin to "being abducted" in that an object retains its identity in name only. Something transformational happens as a result of being decorated, which is done at define time, not at runtime.

In [36]:
@Compose
def m(x): return x + 2

@Compose
def n(x): return 2 * x

In [37]:
(m * n)(10)

22

In [38]:
(n * m)(10)

24

In [52]:
(n * n * m * m)(10)

56

Exercise:  implement ```__pow__(self, n)``` inside Compose so that you can write expressions like:

```python
(n ** 3)(10) # same as (n * n * n)(10)
```

## Properties, Class and Static Methods

Now that you have seen decorator syntax, we can make sense of some additional syntax.

The property decorator allows a method to disguise itself as simply an attribute. 

Different methods apply for getting (shown) and setting (not shown) a property.

Think of attributes as private in the sense that you wish to protect them with code.

In [46]:
from datetime import date

class Animal:

    @staticmethod
    def version(): # no implied leftmost argument, either self or klass
        return "Version 1.0"

    def __init__(self, dob):
        self.dob = dob
        self.stomach = []

    @property
    def age(self):
        return (date.today() - self.dob).days

    def eat(self, food):
        self.stomach.append(food)

    def __repr__(self):
        return "Animal born {}".format(self.dob)

In [40]:
dog = Animal(date(2010, 3, 4))

In [41]:
dog

Animal born 2010-03-04

In [42]:
dog.dob.isoformat()

'2010-03-04'

In [45]:
dog.age

5384

The classmethod decorator allows a method to accept the class itself in place of self, such that the method is about doing something to the class, not the instances directly.

The placeholder leftmost argument is often "klass", spelled that way to avoid name collision with the `class` keyword.

In [22]:
import random 

class Dog(Animal):
    
    tricks = ["roll over", "play dead", "fetch stick"]
    
    def __init__(self, name, dob):
        self.name = name
        super().__init__(dob) # pass dob to superclass __init__

    def do_trick(self):
        return random.choice(self.tricks) # self.tricks checks self.__dict__ then class.__dict__
        
    @classmethod
    def add_trick(klass, trick):  # add a trick to the class-level list named tricks
        klass.tricks.append(trick)

In [23]:
rover = Dog("Rover", date(2000, 1, 10))

In [24]:
rover.age

9089

In [25]:
rover.do_trick()

'roll over'

In [26]:
rover.do_trick()

'fetch stick'

In [27]:
rover.add_trick("beg")

In [28]:
Dog.tricks

['roll over', 'play dead', 'fetch stick', 'beg']

## File I/O and Context Managers

Reading and writing files is a core activity in Python, centered around the builtin functions ```open()``` and ```close()```.

Python's default mode when opening files is to treat them as [UTF-8 text files](https://peps.python.org/pep-0686/). This default encoding may be altered for reading and writing purposes. Sometimes we might like to get a raw bytecode view.

A context manager is a type of object that implements ```__enter__``` and ```__exit__``` which is useful for entering and exiting a "context" wherein the environment has changed for a given block of code, and will be restored when the block is done.  The context might therefore be "with a file open" i.e. "do this block of code with a certain open file, close it when block ends".

For a different take on files and databases, check [Thinking Python section 13](https://allendowney.github.io/ThinkPython/chap13.html).

In [8]:
from pathlib import Path
import os

In [11]:
os.path.isfile("fishes.txt")

True

In [46]:
f = Path("fishes.txt")
f

PosixPath('fishes.txt')

In [47]:
the_file = open(f, 'r')
contents = the_file.read()
the_file.close()

In [48]:
the_file.encoding

'UTF-8'

In [14]:
contents

'Orca is a kind of Dolphin.\nBlue Whale is the largest animal known on earth.\nSharks are the sister group to the Rays (batoids).\nThe Tuna Fish can weigh up to 260 kg.\nSquid and Octopus are in the same class.'

In [15]:
print(contents)

Orca is a kind of Dolphin.
Blue Whale is the largest animal known on earth.
Sharks are the sister group to the Rays (batoids).
The Tuna Fish can weigh up to 260 kg.
Squid and Octopus are in the same class.


In [16]:
with open(f, 'r') as the_file:
    the_file.read()
print(contents)

Orca is a kind of Dolphin.
Blue Whale is the largest animal known on earth.
Sharks are the sister group to the Rays (batoids).
The Tuna Fish can weigh up to 260 kg.
Squid and Octopus are in the same class.


In [29]:
the_file.closed # didn't have to explicitly close it

True

In [30]:
foods = "".join([chr(codepoint) for codepoint in range(127815, 127815 + 50)])

In [31]:
foods

'🍇🍈🍉🍊🍋🍌🍍🍎🍏🍐🍑🍒🍓🍔🍕🍖🍗🍘🍙🍚🍛🍜🍝🍞🍟🍠🍡🍢🍣🍤🍥🍦🍧🍨🍩🍪🍫🍬🍭🍮🍯🍰🍱🍲🍳🍴🍵🍶🍷🍸'

In [32]:
file_object = open("food_emoji.txt", "w")
print(foods, file = file_object)
file_object.close()

In [49]:
file_object = open("food_emoji.txt", "rb") # bytes mode, no encoding
the_bytes=file_object.read()
the_bytes

b'\xf0\x9f\x8d\x87\xf0\x9f\x8d\x88\xf0\x9f\x8d\x89\xf0\x9f\x8d\x8a\xf0\x9f\x8d\x8b\xf0\x9f\x8d\x8c\xf0\x9f\x8d\x8d\xf0\x9f\x8d\x8e\xf0\x9f\x8d\x8f\xf0\x9f\x8d\x90\xf0\x9f\x8d\x91\xf0\x9f\x8d\x92\xf0\x9f\x8d\x93\xf0\x9f\x8d\x94\xf0\x9f\x8d\x95\xf0\x9f\x8d\x96\xf0\x9f\x8d\x97\xf0\x9f\x8d\x98\xf0\x9f\x8d\x99\xf0\x9f\x8d\x9a\xf0\x9f\x8d\x9b\xf0\x9f\x8d\x9c\xf0\x9f\x8d\x9d\xf0\x9f\x8d\x9e\xf0\x9f\x8d\x9f\xf0\x9f\x8d\xa0\xf0\x9f\x8d\xa1\xf0\x9f\x8d\xa2\xf0\x9f\x8d\xa3\xf0\x9f\x8d\xa4\xf0\x9f\x8d\xa5\xf0\x9f\x8d\xa6\xf0\x9f\x8d\xa7\xf0\x9f\x8d\xa8\xf0\x9f\x8d\xa9\xf0\x9f\x8d\xaa\xf0\x9f\x8d\xab\xf0\x9f\x8d\xac\xf0\x9f\x8d\xad\xf0\x9f\x8d\xae\xf0\x9f\x8d\xaf\xf0\x9f\x8d\xb0\xf0\x9f\x8d\xb1\xf0\x9f\x8d\xb2\xf0\x9f\x8d\xb3\xf0\x9f\x8d\xb4\xf0\x9f\x8d\xb5\xf0\x9f\x8d\xb6\xf0\x9f\x8d\xb7\xf0\x9f\x8d\xb8\n'

In [35]:
file_object.close()

In [54]:
with open("food_emoji.txt", "rt") as file_object:
    print(file_object.read())

🍇🍈🍉🍊🍋🍌🍍🍎🍏🍐🍑🍒🍓🍔🍕🍖🍗🍘🍙🍚🍛🍜🍝🍞🍟🍠🍡🍢🍣🍤🍥🍦🍧🍨🍩🍪🍫🍬🍭🍮🍯🍰🍱🍲🍳🍴🍵🍶🍷🍸



In [55]:
file_object.encoding

'UTF-8'

In [50]:
with open("food_emoji.txt", "rb") as file_object:
    print(file_object.read())

b'\xf0\x9f\x8d\x87\xf0\x9f\x8d\x88\xf0\x9f\x8d\x89\xf0\x9f\x8d\x8a\xf0\x9f\x8d\x8b\xf0\x9f\x8d\x8c\xf0\x9f\x8d\x8d\xf0\x9f\x8d\x8e\xf0\x9f\x8d\x8f\xf0\x9f\x8d\x90\xf0\x9f\x8d\x91\xf0\x9f\x8d\x92\xf0\x9f\x8d\x93\xf0\x9f\x8d\x94\xf0\x9f\x8d\x95\xf0\x9f\x8d\x96\xf0\x9f\x8d\x97\xf0\x9f\x8d\x98\xf0\x9f\x8d\x99\xf0\x9f\x8d\x9a\xf0\x9f\x8d\x9b\xf0\x9f\x8d\x9c\xf0\x9f\x8d\x9d\xf0\x9f\x8d\x9e\xf0\x9f\x8d\x9f\xf0\x9f\x8d\xa0\xf0\x9f\x8d\xa1\xf0\x9f\x8d\xa2\xf0\x9f\x8d\xa3\xf0\x9f\x8d\xa4\xf0\x9f\x8d\xa5\xf0\x9f\x8d\xa6\xf0\x9f\x8d\xa7\xf0\x9f\x8d\xa8\xf0\x9f\x8d\xa9\xf0\x9f\x8d\xaa\xf0\x9f\x8d\xab\xf0\x9f\x8d\xac\xf0\x9f\x8d\xad\xf0\x9f\x8d\xae\xf0\x9f\x8d\xaf\xf0\x9f\x8d\xb0\xf0\x9f\x8d\xb1\xf0\x9f\x8d\xb2\xf0\x9f\x8d\xb3\xf0\x9f\x8d\xb4\xf0\x9f\x8d\xb5\xf0\x9f\x8d\xb6\xf0\x9f\x8d\xb7\xf0\x9f\x8d\xb8\n'


In [40]:
foods.encode('utf-8')

b'\xf0\x9f\x8d\x87\xf0\x9f\x8d\x88\xf0\x9f\x8d\x89\xf0\x9f\x8d\x8a\xf0\x9f\x8d\x8b\xf0\x9f\x8d\x8c\xf0\x9f\x8d\x8d\xf0\x9f\x8d\x8e\xf0\x9f\x8d\x8f\xf0\x9f\x8d\x90\xf0\x9f\x8d\x91\xf0\x9f\x8d\x92\xf0\x9f\x8d\x93\xf0\x9f\x8d\x94\xf0\x9f\x8d\x95\xf0\x9f\x8d\x96\xf0\x9f\x8d\x97\xf0\x9f\x8d\x98\xf0\x9f\x8d\x99\xf0\x9f\x8d\x9a\xf0\x9f\x8d\x9b\xf0\x9f\x8d\x9c\xf0\x9f\x8d\x9d\xf0\x9f\x8d\x9e\xf0\x9f\x8d\x9f\xf0\x9f\x8d\xa0\xf0\x9f\x8d\xa1\xf0\x9f\x8d\xa2\xf0\x9f\x8d\xa3\xf0\x9f\x8d\xa4\xf0\x9f\x8d\xa5\xf0\x9f\x8d\xa6\xf0\x9f\x8d\xa7\xf0\x9f\x8d\xa8\xf0\x9f\x8d\xa9\xf0\x9f\x8d\xaa\xf0\x9f\x8d\xab\xf0\x9f\x8d\xac\xf0\x9f\x8d\xad\xf0\x9f\x8d\xae\xf0\x9f\x8d\xaf\xf0\x9f\x8d\xb0\xf0\x9f\x8d\xb1\xf0\x9f\x8d\xb2\xf0\x9f\x8d\xb3\xf0\x9f\x8d\xb4\xf0\x9f\x8d\xb5\xf0\x9f\x8d\xb6\xf0\x9f\x8d\xb7\xf0\x9f\x8d\xb8'

## Context manager with Decimal 

In [102]:
import decimal
from decimal import Decimal, localcontext, getcontext

In [100]:
localcontext

Context(prec=100, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[InvalidOperation, FloatOperation], traps=[InvalidOperation, DivisionByZero, Overflow])

In [101]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, FloatOperation, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])

In [125]:
class CM:  

    def __init__(self, prec=6):
        self.prec = prec

    def __enter__(self, prec=6):
        print("entering context")
        self.old_prec = getcontext().prec
        getcontext().prec = self.prec
        return getcontext()

    def __exit__(self, *oops):
        print("exiting context")
        decimal.getcontext().prec = self.old_prec   

In [126]:
print(Decimal(1)/Decimal(7))

0.1428571428571428571428571429


In [130]:
with CM() as ctx:
    print(ctx)
    print(Decimal(1)/Decimal(7))

entering context
Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, FloatOperation, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])
0.142857
exiting context


In [131]:
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, FloatOperation, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])

## Decorator: contextlib.contextmanager

The enter/exit protocol is similar to running code up to yield, then finishing, in a generator function. The standard library provides a decorator that will turn a simple function generator into a context manager. The ```__enter__``` part runs up the the ```yield``` after which the ```exit``` part runs when the generator resumes.

In [113]:
from contextlib import contextmanager

In [138]:
@contextmanager
def cm(p):
    print("entering context")
    old_prec = getcontext().prec
    getcontext().prec = p
    yield getcontext()
    print("exiting context")
    decimal.getcontext().prec = old_prec     

In [139]:
print(Decimal(1)/Decimal(7))

0.1428571428571428571428571429


In [140]:
with cm(6) as ctx:
    print(ctx)
    print(Decimal(1)/Decimal(7))

entering context
Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, FloatOperation, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])
0.142857
exiting context


In [141]:
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, FloatOperation, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])